In [3]:
import tensorflow as tf

<h1>tf.Variable vs tf.get_variable</h1>
<p style="font-size:20px"><b>tf.Variable</b> is a constructor that creates new variable each time you call it.

In [4]:
tf.reset_default_graph()
W = tf.Variable([1,2,3],name='W')
W1 = tf.Variable([4,5,6],name='W')
print(W.name)
print(W1.name)
print(W==W1)

W:0
W_1:0
False


<p style="font-size:20px"><b>tf.get_variable</b> will create a new variable if it hasn't been created before. If it exists, it will return an error if you try to create another one with the same name again.

In [5]:
tf.reset_default_graph()
W = tf.get_variable('W',shape=[3],dtype=tf.float32)
b = tf.get_variable('W',shape=[],dtype=tf.float32)
print(W.name)
print(b.name)
print(W==b)

ValueError: Variable W already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-5-01bb05a3f7b9>", line 2, in <module>
    W = tf.get_variable('W',shape=[3],dtype=tf.float32)
  File "/home/sukun/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/sukun/anaconda3/envs/tensorflow_gpuenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3183, in run_ast_nodes
    if (yield from self.run_code(code, result)):


<p style="font-size:20px">To share or reuse the variables, we need to understand the name scopes and variable scopes.

<h1>Name Scopes</h1>

<p style="font-size:20px">When deal with more complex models such as neural networks, the graph can easily become cluttered with thousands of nodes. To avoid this, you can create name scopes to group related nodes.

In [13]:
tf.reset_default_graph()
with tf.name_scope("my_scope"):
    W1 = tf.get_variable("W1",[1],dtype=tf.float32)
    W2 = tf.Variable(1,name="W2",dtype=tf.float32)
    a = tf.add(W1,W2,name="add")

print(W1.name)
print(W2.name)
print(a.name)

W1:0
my_scope/W2:0
my_scope/add:0


<p style="font-size:20px">We can see name scope is ignored by <b>tf.get_variable</b>. The only way to place a variable accessed using <b>tf.get_variable</b> in a scope is to use a <b>variable scope</b> (We recommend everyone to use <b>variable_scope</b> and not use <b>name_scope</b> except for internal code and libraries), as in the following example:

<h1>Variable Scopes</h1>

In [14]:
tf.reset_default_graph()
with tf.variable_scope("my_scope"):
    W1 = tf.get_variable("W1",[1],dtype=tf.float32)
    W2 = tf.Variable(1,name="W2",dtype=tf.float32)
    a = tf.add(W1,W2,name="add")

print(W1.name)
print(W2.name)
print(a.name)    

my_scope/W1:0
my_scope/W2:0
my_scope/add:0


<p style="font-size:20px">This allows us to easily share variables across different parts of the program, even within different name scopes

<h1>Sharing Variables</h1>

In [16]:
tf.reset_default_graph()
with tf.name_scope("foo"):
    with tf.variable_scope("scope_1"):
        W = tf.get_variable("W",[1])
with tf.name_scope("bar"):
    with tf.variable_scope("scope_1",reuse=True):
        b = tf.get_variable("W",[1])

assert W==b
print(W.name)
print(b.name)

scope_1/W:0
scope_1/W:0


<h1>AND Gate Example</h1>

In [ ]:
tf.reset_default_graph()
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "AND_logs_scope"
logdir = "{}/run-{}/".format(root_logdir,now)

#define number of training iterations
NUM_ITER = 2000
#define learning rate
lr = 0.01

#define the training input data
x = np.array([[0,0],[1,0],[1,1],[0,1]],np.float32)
#define the target outputs/labels for AND gate
y = np.array([0,0,1,0],np.float32)
#y = np.array([0,1,1,1],np.float32)       (uncomment this line if you are doing OR gate)
y = np.reshape(y,[4,1])

with tf.variable_scope('data'):
    #define the placeholder to feed the training data later
    X = tf.placeholder(tf.float32,shape=[4,2],name='X')
    Y = tf.placeholder(tf.float32,shape=[4,1],name='Y')

with tf.variable_scope("Parameters"):
    #define the weights and initialized with random normal distribution
    W = tf.Variable(tf.random_normal([2,1]),tf.float32,name='Weight')
    #define the bias with zero initialization
    b = tf.Variable(tf.zeros([1,1]),tf.float32,name='bias')

with tf.variable_scope("Prediction"):
    #define the predicted output
    Y_hat = tf.add(tf.matmul(X,W),b,'Y_hat')
with tf.variable_scope("Loss"):
    #define the loss function, we are using cross entropy function after sigmoid on Y_hat
    loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=Y_hat,labels=Y),name='loss')

#define the optimizer, here we use standard gradient descent method
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)

#initialize tensorflow variables
init = tf.global_variables_initializer()

#!!!!!!!!!!!!!define loss scalar summary & File Writer!!!!!!!!
with tf.name_scope('summaries'):
    loss_summary = tf.summary.scalar('loss',loss)

file_writer = tf.summary.FileWriter(logdir,tf.get_default_graph())

#start tensorflow session
with tf.Session() as sess:
    #run initialization
    sess.run(init)
    #training for number of iterations
    for i in range(NUM_ITER):
        if i%10 == 0:
            #fetch the loss every 10 iterations
            summary = sess.run(loss_summary,feed_dict={X:x,Y:y})
            #add the summary to file writer
            file_writer.add_summary(summary,i)
        #every iteration we run optimization on the training data X and labels Y
        sess.run(optimizer,feed_dict = {X:x,Y:y})
        
    #after training, we acquire the weight and bias
    #np.squeeze plays a role to get rid of the redundant dimension [i.e.make (2,1) be (2,)]
    W = np.squeeze(sess.run(W))
    b = np.squeeze(sess.run(b))

#remember to close file writer!
file_writer.close()

<h2>Open Tensorboard you would see the clearer graph sctructure</h2>

<img src="figures/name_scope.png">